In [3]:
import pandas as pd
import numpy as np
#import json
from google.cloud import bigquery

In [4]:
#Localisation du projet name et table_id sur gcp
project="marbotic"

In [5]:
#intégration des credentials 
from google.oauth2 import service_account

key_path = "/Users/antonin/code/AntoninAnq/gcp/marbotic-7d02fac30bd8.json"

credentials = service_account.Credentials.from_service_account_file(
    key_path, scopes=["https://www.googleapis.com/auth/cloud-platform"],
)

In [6]:
def extract(project,credentials):
    # Construct a BigQuery client object.
    client = bigquery.Client(project=project, credentials=credentials)

    query = """
    SELECT
    COUNT(DISTINCT ep.Activity_Name) as unique_activity, 
    sm.session_id,any_value(Time_Spent) as Time_Spent,
    any_value(Action_Element_Name) as Action_Element_Name,
    any_value(Profile_Age_days) as Profile_Age_days,
    (SAFE_DIVIDE(SUM(event_type_Activity_Good_Answer) , (SUM(event_type_Activity_Good_Answer)
    + SUM(event_type_Activity_Wrong_Answer))))*100 as good_answer_ratio
    FROM `marbotic.marbotic_dataset.event_properties` as ep
    JOIN `marbotic.marbotic_dataset.sessions_metrics` as sm on ep.session_id = sm.session_id
    GROUP BY sm.session_id
    ORDER BY unique_activity DESC
    """

    query_job = client.query(query)  # Make an API request.
    results=query_job.result().to_dataframe() # Waits for job to complete.

    return pd.DataFrame(results)

In [7]:
#On récupère d'event le nombre d'activity unique par session, le ratio de bonne réponse par sessions et le time spent par sessions
first_data = extract(project,credentials)
first_data

,unique_activity,session_id,Time_Spent,Action_Element_Name,Profile_Age_days,good_answer_ratio
0,42,1.648054e+12,8.0,Back,1899.0,100.0
1,41,1.646647e+12,1412.0,Back,8434.0,NaN
2,40,1.646297e+12,19.0,Back,1968.0,100.0
3,39,1.647908e+12,3.0,Back,2376.0,100.0
4,38,1.646244e+12,167.0,Back,2227.0,100.0
...,...,...,...,...,...,...
18811,0,1.649372e+12,6187.0,None,NaN,NaN
18812,0,1.649440e+12,243.0,None,NaN,NaN
18813,0,1.649485e+12,801.0,None,NaN,NaN
18814,0,1.649650e+12,924.0,None,NaN,NaN


In [7]:
def extract_session_metrics(project,credentials):
    # Construct a BigQuery client object.
    client = bigquery.Client(project=project, credentials=credentials)

    query = """
    SELECT *
    FROM `marbotic.marbotic_dataset.sessions_metrics`
    """

    query_job = client.query(query)  # Make an API request.
    results=query_job.result().to_dataframe() # Waits for job to complete.

    return pd.DataFrame(results)

In [8]:
session_metrics_df = extract_session_metrics(project,credentials)
session_metrics_df

,session_id,city,language,os_version,region,country,device_model,start_version,session_end,id,...,event_type_Profile_Update,event_type_Scaffolding_Reset,event_type_Scaffolding_Scaffolding,event_type_Scene_Enter,event_type_Scene_Leave,event_type_Toast_Appear,event_type_Toast_Disappear,session_start,game_events,setup_events
0,1.647421e+12,Certines,French,15.3.1,Ain,France,"iPad11,6",1.0.1,2022-03-16 08:58:14.214000+00:00,6,...,0.0,0.0,0.0,5.0,1.0,0.0,0.0,2022-03-16 08:56:40.722000+00:00,6.0,0.0
1,1.647421e+12,Certines,French,15.1,Ain,France,"iPad11,6",1.0.1,2022-03-16 08:58:45.380000+00:00,5,...,0.0,0.0,0.0,4.0,1.0,0.0,0.0,2022-03-16 08:58:28.269000+00:00,5.0,0.0
2,1.648134e+12,Certines,French,15.3.1,Ain,France,"iPad11,6",1.0.1,2022-03-24 15:06:35.838000+00:00,5,...,0.0,0.0,0.0,1.0,1.0,0.0,0.0,2022-03-24 15:06:21.426000+00:00,5.0,0.0
3,1.647421e+12,Certines,French,15.1,Ain,France,"iPad11,6",1.0.1,2022-03-16 08:57:59.845000+00:00,5,...,0.0,0.0,0.0,4.0,1.0,0.0,0.0,2022-03-16 08:57:48.381000+00:00,5.0,0.0
4,1.646641e+12,Izernore,French,15.2,Ain,France,iPad 6,1.0.3,2022-03-07 08:25:39.783000+00:00,6,...,0.0,0.0,0.0,1.0,1.0,0.0,0.0,2022-03-07 08:24:40.937000+00:00,6.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
18811,1.646491e+12,Notting Hill Gate,English,15.1,Royal Kensington and Chelsea,United Kingdom,iPad 6,1.0.7,2022-03-05 14:40:14.517000+00:00,5,...,0.0,0.0,0.0,4.0,1.0,0.0,0.0,2022-03-05 14:39:48.894000+00:00,5.0,0.0
18812,1.652027e+12,Hamburg,English,14.6,Free and Hanseatic City of Hamburg,Germany,iPad Pro,4.1.3,2022-05-08 16:39:55.181000+00:00,17,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2022-05-08 16:32:21.326000+00:00,0.0,0.0
18813,1.652029e+12,Hamburg,English,14.6,Free and Hanseatic City of Hamburg,Germany,iPad Pro,4.1.3,2022-05-08 17:01:24.501000+00:00,20,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2022-05-08 16:58:54.378000+00:00,0.0,0.0
18814,1.652377e+12,Hamburg,English,14.6,Free and Hanseatic City of Hamburg,Germany,iPad Pro,4.1.3,2022-05-12 18:05:34.559000+00:00,19,...,0.0,0.0,0.0,10.0,4.0,0.0,0.0,2022-05-12 17:38:47.898000+00:00,19.0,0.0


In [9]:
#on merge les première colonnes crées avec la table sessions metrics
sm_df = session_metrics_df.merge(first_data, on='session_id')

In [10]:
#Calcul du temps de chaque session
sm_df['duration_min'] = (sm_df['session_end']-sm_df['session_start']).apply(lambda x: x.total_seconds()/60)

In [11]:
sm_df

,session_id,city,language,os_version,region,country,device_model,start_version,session_end,id,...,setup_events,unique_activity,Time_Spent,Action_Element_Name,Profile_Age_days,Error_Type,Activity_Solved,Scaffolding_Level,good_answer_ratio,duration_min
0,1.647421e+12,Certines,French,15.3.1,Ain,France,"iPad11,6",1.0.1,2022-03-16 08:58:14.214000+00:00,6,...,0.0,0,5.0,None,-1.0,None,None,NaN,NaN,1.558200
1,1.647421e+12,Certines,French,15.1,Ain,France,"iPad11,6",1.0.1,2022-03-16 08:58:45.380000+00:00,5,...,0.0,0,4.0,None,-1.0,None,None,NaN,NaN,0.285183
2,1.648134e+12,Certines,French,15.3.1,Ain,France,"iPad11,6",1.0.1,2022-03-24 15:06:35.838000+00:00,5,...,0.0,0,7.0,None,-1.0,None,None,NaN,NaN,0.240200
3,1.647421e+12,Certines,French,15.1,Ain,France,"iPad11,6",1.0.1,2022-03-16 08:57:59.845000+00:00,5,...,0.0,0,5.0,None,-1.0,None,None,NaN,NaN,0.191067
4,1.646641e+12,Izernore,French,15.2,Ain,France,iPad 6,1.0.3,2022-03-07 08:25:39.783000+00:00,6,...,0.0,0,247.0,Back,1891.0,None,None,NaN,NaN,0.980767
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
18811,1.646491e+12,Notting Hill Gate,English,15.1,Royal Kensington and Chelsea,United Kingdom,iPad 6,1.0.7,2022-03-05 14:40:14.517000+00:00,5,...,0.0,0,5.0,None,-1.0,None,None,NaN,NaN,0.427050
18812,1.652027e+12,Hamburg,English,14.6,Free and Hanseatic City of Hamburg,Germany,iPad Pro,4.1.3,2022-05-08 16:39:55.181000+00:00,17,...,0.0,0,52.0,None,4930.0,None,None,NaN,NaN,7.564250
18813,1.652029e+12,Hamburg,English,14.6,Free and Hanseatic City of Hamburg,Germany,iPad Pro,4.1.3,2022-05-08 17:01:24.501000+00:00,20,...,0.0,1,33.0,None,4930.0,None,None,0.0,NaN,2.502050
18814,1.652377e+12,Hamburg,English,14.6,Free and Hanseatic City of Hamburg,Germany,iPad Pro,4.1.3,2022-05-12 18:05:34.559000+00:00,19,...,0.0,1,1371.0,Back,4934.0,None,False,16.0,100.0,26.777683


In [12]:
def extract_user_metrics(project,credentials):
    # Construct a BigQuery client object.
    client = bigquery.Client(project=project, credentials=credentials)

    query = """
    SELECT user_creation_time, user_id
    FROM `marbotic.marbotic_dataset.users_metrics`
    """

    query_job = client.query(query)  # Make an API request.
    results=query_job.result().to_dataframe() # Waits for job to complete.

    return pd.DataFrame(results)

In [13]:
user_metrics_df = extract_user_metrics(project,credentials)

In [14]:
#on merge la date de création dans le cas ou on veut calculer la maturité du user au moment de la session. Non utilisé dans un premier temps
sm_df = sm_df.merge(user_metrics_df, on='user_id')

In [15]:
sm_df

,session_id,city,language,os_version,region,country,device_model,start_version,session_end,id,...,unique_activity,Time_Spent,Action_Element_Name,Profile_Age_days,Error_Type,Activity_Solved,Scaffolding_Level,good_answer_ratio,duration_min,user_creation_time
0,1.647421e+12,Certines,French,15.3.1,Ain,France,"iPad11,6",1.0.1,2022-03-16 08:58:14.214000+00:00,6,...,0,5.0,None,-1.0,None,None,NaN,NaN,1.558200,2021-11-20 15:17:18.537000+00:00
1,1.647421e+12,Certines,French,15.1,Ain,France,"iPad11,6",1.0.1,2022-03-16 08:58:45.380000+00:00,5,...,0,4.0,None,-1.0,None,None,NaN,NaN,0.285183,2021-11-20 15:17:18.537000+00:00
2,1.648134e+12,Certines,French,15.3.1,Ain,France,"iPad11,6",1.0.1,2022-03-24 15:06:35.838000+00:00,5,...,0,7.0,None,-1.0,None,None,NaN,NaN,0.240200,2021-11-20 15:17:18.537000+00:00
3,1.647421e+12,Certines,French,15.1,Ain,France,"iPad11,6",1.0.1,2022-03-16 08:57:59.845000+00:00,5,...,0,5.0,None,-1.0,None,None,NaN,NaN,0.191067,2021-11-20 15:17:18.537000+00:00
4,1.646641e+12,Izernore,French,15.2,Ain,France,iPad 6,1.0.3,2022-03-07 08:25:39.783000+00:00,6,...,0,247.0,Back,1891.0,None,None,NaN,NaN,0.980767,2021-12-07 18:08:20.505000+00:00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
18811,1.649501e+12,Kensington,English,15.1,Royal Kensington and Chelsea,United Kingdom,"iPad12,1",1.0.5,2022-04-09 10:45:26.022000+00:00,1,...,1,3573.0,None,NaN,None,None,NaN,NaN,0.000000,2021-12-19 14:16:39.549000+00:00
18812,1.652027e+12,Hamburg,English,14.6,Free and Hanseatic City of Hamburg,Germany,iPad Pro,4.1.3,2022-05-08 16:39:55.181000+00:00,17,...,0,52.0,None,4930.0,None,None,NaN,NaN,7.564250,2022-05-08 16:29:14.356000+00:00
18813,1.652029e+12,Hamburg,English,14.6,Free and Hanseatic City of Hamburg,Germany,iPad Pro,4.1.3,2022-05-08 17:01:24.501000+00:00,20,...,1,33.0,None,4930.0,None,None,0.0,NaN,2.502050,2022-05-08 16:29:14.356000+00:00
18814,1.652377e+12,Hamburg,English,14.6,Free and Hanseatic City of Hamburg,Germany,iPad Pro,4.1.3,2022-05-12 18:05:34.559000+00:00,19,...,1,1371.0,Back,4934.0,None,False,16.0,100.0,26.777683,2022-05-08 16:29:14.356000+00:00


In [16]:
#On calcul le nombre de sessions avant celle en cours
def nb_session(user_id,session_start):
    return sm_df[(sm_df['user_id']==user_id) & (sm_df['session_start']<session_start)].shape[0]

In [17]:
sm_df['previous_session'] = sm_df.apply(lambda x:nb_session(x.user_id,x.session_start), axis=1)

In [18]:
#On calcul le temps depuis la dernière sessions
def last_session_time(user_id,session_start):
    last_session = sm_df[(sm_df['user_id']==user_id) & (sm_df['session_start']<session_start)]\
    .groupby('user_id').last()['session_start']
    if len(last_session.values)==0:
        return 0
    else:
        last_session = last_session.dt.to_pydatetime()[0].replace(tzinfo=None)
        return (session_start.to_pydatetime().replace(tzinfo=None) - last_session)

In [19]:
(sm_df[(sm_df['user_id']==65616.0) & (sm_df['session_start']<'2022-03-16 08:58:28.269000+00:00')].groupby('user_id').last()['session_start'].dt.to_pydatetime()[0].replace(tzinfo=None))

datetime.datetime(2022, 3, 16, 8, 57, 48, 381000)

In [20]:
sm_df['time_since_last_session'] = sm_df.apply(lambda x:last_session_time(x.user_id,x.session_start), axis=1)

In [21]:
sm_df

,session_id,city,language,os_version,region,country,device_model,start_version,session_end,id,...,Action_Element_Name,Profile_Age_days,Error_Type,Activity_Solved,Scaffolding_Level,good_answer_ratio,duration_min,user_creation_time,previous_session,time_since_last_session
0,1.647421e+12,Certines,French,15.3.1,Ain,France,"iPad11,6",1.0.1,2022-03-16 08:58:14.214000+00:00,6,...,None,-1.0,None,None,NaN,NaN,1.558200,2021-11-20 15:17:18.537000+00:00,0,0
1,1.647421e+12,Certines,French,15.1,Ain,France,"iPad11,6",1.0.1,2022-03-16 08:58:45.380000+00:00,5,...,None,-1.0,None,None,NaN,NaN,0.285183,2021-11-20 15:17:18.537000+00:00,2,0:00:39.888000
2,1.648134e+12,Certines,French,15.3.1,Ain,France,"iPad11,6",1.0.1,2022-03-24 15:06:35.838000+00:00,5,...,None,-1.0,None,None,NaN,NaN,0.240200,2021-11-20 15:17:18.537000+00:00,3,"8 days, 6:08:33.045000"
3,1.647421e+12,Certines,French,15.1,Ain,France,"iPad11,6",1.0.1,2022-03-16 08:57:59.845000+00:00,5,...,None,-1.0,None,None,NaN,NaN,0.191067,2021-11-20 15:17:18.537000+00:00,1,0:01:07.659000
4,1.646641e+12,Izernore,French,15.2,Ain,France,iPad 6,1.0.3,2022-03-07 08:25:39.783000+00:00,6,...,Back,1891.0,None,None,NaN,NaN,0.980767,2021-12-07 18:08:20.505000+00:00,5,"1 day, 16:24:40.115000"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
18811,1.649501e+12,Kensington,English,15.1,Royal Kensington and Chelsea,United Kingdom,"iPad12,1",1.0.5,2022-04-09 10:45:26.022000+00:00,1,...,None,NaN,None,None,NaN,NaN,0.000000,2021-12-19 14:16:39.549000+00:00,2,17:42:45.614000
18812,1.652027e+12,Hamburg,English,14.6,Free and Hanseatic City of Hamburg,Germany,iPad Pro,4.1.3,2022-05-08 16:39:55.181000+00:00,17,...,None,4930.0,None,None,NaN,NaN,7.564250,2022-05-08 16:29:14.356000+00:00,0,0
18813,1.652029e+12,Hamburg,English,14.6,Free and Hanseatic City of Hamburg,Germany,iPad Pro,4.1.3,2022-05-08 17:01:24.501000+00:00,20,...,None,4930.0,None,None,0.0,NaN,2.502050,2022-05-08 16:29:14.356000+00:00,1,0:26:33.052000
18814,1.652377e+12,Hamburg,English,14.6,Free and Hanseatic City of Hamburg,Germany,iPad Pro,4.1.3,2022-05-12 18:05:34.559000+00:00,19,...,Back,4934.0,None,False,16.0,100.0,26.777683,2022-05-08 16:29:14.356000+00:00,2,"4 days, 0:39:53.520000"
